In [1]:
import os, sys, copy

import numpy as np
from tabulate import tabulate

from helper import visualize_tabulation, all_slack_starting, dual_simplex_to_all_slack_starting

### Define the LP Problem

In [2]:
# no. of decision variables (N)
NO_OF_DECISION_VARS = 4

# no. of conditions 
NO_OF_CONDITIONS = 3

# --------------------------------- objective function ---------------------------------
# specify the objective function in the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>]
OBJ_FUNC = [4, 5, 9, 11]

# -------------------------------- type of optimization --------------------------------
OPT_TYPE = "MAX" # or "MIN" 

# ------------------------------------- conditions -------------------------------------
# specify each condition in a new row with the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>, <solution>]
conditions = [
    [  1,   1,   1,   1,  15],
    [  7,   5,   3,   2, 120],
    [  3,   5,  10,  15, 100],
]

### Create the Variables and the Initial Tabulation

In [3]:
var_symbol_arr = []
basic_var_symbol_arr = []

OBJ_FUNC_VAR_SYMBOL = "P"
DECISION_VAR_SYMBOL = "X"
SLACK_VAR_SYMBOL    = "S"

# ========================= ALL THE VARIABLES =========================
var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_DECISION_VARS):
    var_symbol_arr.append(DECISION_VAR_SYMBOL + str(i+1))

for i in range(NO_OF_CONDITIONS):
    var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("variables:", var_symbol_arr)

# ========================== BASIC VARIABLES ==========================
basic_var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_CONDITIONS):
    basic_var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("basic variables:", basic_var_symbol_arr)

variables: ['P', 'X1', 'X2', 'X3', 'X4', 'S1', 'S2', 'S3']
basic variables: ['P', 'S1', 'S2', 'S3']


In [4]:
# verify the elements
OBJ_FUNC = np.array(OBJ_FUNC)
if len(OBJ_FUNC) != NO_OF_DECISION_VARS:
    raise Exception(f"the no. of coefficients in the objective function doesn't match tne number of decision variables...")

conditions = np.array(conditions)
if len(conditions) != NO_OF_CONDITIONS:
    raise Exception(f"the no. of specified conditions doesn't match 'NO_OF_CONDITIONS'...")
if len(conditions[0]) != NO_OF_DECISION_VARS + 1:
    raise Exception(f"the no. of coefficients in the conditions doesn't match tne number of decision variables...")

In [5]:
# find the size of a row
row_size = 1 + NO_OF_DECISION_VARS + NO_OF_CONDITIONS + 1

# find the column size
col_size = 1 + NO_OF_CONDITIONS # no. of basic vars = 1 (for objective function) + no. of conditions

# construct the initial tabulation
obj_func_row = np.concatenate(([1], -OBJ_FUNC, np.zeros([NO_OF_CONDITIONS]), [0]))
initial_tabulation = obj_func_row.reshape((1, -1))

for i in range(1, col_size):

    # condition rows
    temp = np.zeros([NO_OF_CONDITIONS])
    temp[i-1] = 1
    row = np.concatenate(([0], conditions[i-1][ :-1], temp, conditions[i-1][-1: ])).reshape((1, -1))

    initial_tabulation = np.concatenate((initial_tabulation, row), axis=0)

visualize_tabulation(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

┌────┬─────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬───────┐
│    │   P │   X1 │   X2 │   X3 │   X4 │   S1 │   S2 │   S3 │   sol │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ P  │   1 │   -4 │   -5 │   -9 │  -11 │    0 │    0 │    0 │     0 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │    1 │    1 │    1 │    1 │    1 │    0 │    0 │    15 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │    7 │    5 │    3 │    2 │    0 │    1 │    0 │   120 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S3 │   0 │    3 │    5 │   10 │   15 │    0 │    0 │    1 │   100 │
└────┴─────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴───────┘


### Perform the All-Slack Starting Method

In [6]:
# obtain the optimal tabulation
optimal_tabulation, basic_var_symbol_arr = all_slack_starting(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, opt_type=OPT_TYPE)

==================== iteration no.: 1 ====================
entering var: X4
pivot col: [-11.   1.   2.  15.]
solution : [  0.  15. 120. 100.]
ratio col: [-0.         15.         60.          6.66666667]
leaving var : S3
pivot row: [  0.   3.   5.  10.  15.   0.   0.   1. 100.]
new basic variables: ['P', 'S1', 'S2', 'X4']
┌────┬─────┬──────┬─────────┬─────────┬──────┬──────┬──────┬─────────┬──────────┐
│    │   P │   X1 │      X2 │      X3 │   X4 │   S1 │   S2 │      S3 │      sol │
├────┼─────┼──────┼─────────┼─────────┼──────┼──────┼──────┼─────────┼──────────┤
│ P  │   1 │ -1.8 │ -1.3333 │ -1.6667 │    0 │    0 │    0 │  0.7333 │  73.3333 │
├────┼─────┼──────┼─────────┼─────────┼──────┼──────┼──────┼─────────┼──────────┤
│ S1 │   0 │  0.8 │  0.6667 │  0.3333 │    0 │    1 │    0 │ -0.0667 │   8.3333 │
├────┼─────┼──────┼─────────┼─────────┼──────┼──────┼──────┼─────────┼──────────┤
│ S2 │   0 │  6.6 │  4.3333 │  1.6667 │    0 │    0 │    1 │ -0.1333 │ 106.667  │
├────┼─────┼──────┼──

**What are the underutilized resources here?** 

- resource 2 (46.4286 extra)

**What happens if we reduce the available material 2 resource by an amount of 60?**

In [8]:
# extract 'contribution margin matrix'
contribution_margin_matrix = optimal_tabulation[:, [0, 5, 6, 7]]

In [19]:
contribution_margin_matrix

array([[ 1.        ,  1.85714286,  0.        ,  0.71428571],
       [ 0.        ,  1.42857143,  0.        , -0.14285714],
       [ 0.        , -8.71428571,  1.        ,  0.57142857],
       [ 0.        , -0.42857143,  0.        ,  0.14285714]])

In [10]:
# find the new solution column in the initial tabulation after the change
new_init_solution = np.array([0, 15, 60, 100]).reshape((-1, 1))

In [14]:
# find the new solution column in the new final optimal tabulation
new_opt_solution = (contribution_margin_matrix @ new_init_solution)
print(f"new optimal solution column: \n{new_opt_solution}")

new optimal solution column: 
[[ 99.28571429]
 [  7.14285714]
 [-13.57142857]
 [  7.85714286]]


In [16]:
# find the new optimal tabulation
new_optimal_tabulation = copy.deepcopy(optimal_tabulation)
new_optimal_tabulation[:, -1] = new_opt_solution.reshape((-1, ))

In [18]:
# apply dual-simplex method to make the tabulation feasible
feasible_new_tabulation, basic_var_symbol_arr = dual_simplex_to_all_slack_starting(new_optimal_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

====================DUAL SIMPLEX - iteration no.: 1====================
leaving variale : S2
ratio row       : [       inf        nan 0.5               nan 0.84615385 0.21311475
 0.         1.25       7.31578947]
entering variale: S1
┌────┬─────┬──────┬────────┬──────┬─────────┬──────┬─────────┬─────────┬─────────┐
│    │   P │   X1 │     X2 │   X3 │      X4 │   S1 │      S2 │      S3 │     sol │
├────┼─────┼──────┼────────┼──────┼─────────┼──────┼─────────┼─────────┼─────────┤
│ P  │   1 │    0 │ 0.2459 │    0 │  1.9672 │    0 │  0.2131 │  0.8361 │ 96.3934 │
├────┼─────┼──────┼────────┼──────┼─────────┼──────┼─────────┼─────────┼─────────┤
│ X1 │   0 │    1 │ 0.5738 │    0 │ -0.4098 │    0 │  0.1639 │ -0.0492 │  4.918  │
├────┼─────┼──────┼────────┼──────┼─────────┼──────┼─────────┼─────────┼─────────┤
│ S1 │  -0 │   -0 │ 0.0984 │   -0 │ -0.2131 │    1 │ -0.1148 │ -0.0656 │  1.5574 │
├────┼─────┼──────┼────────┼──────┼─────────┼──────┼─────────┼─────────┼─────────┤
│ X3 │   0 │    0 │

/home/akila/semester_8/OR/helper.py:148: RuntimeWarning: divide by zero encountered in divide
  ratio_row = np.abs(tabulation[0]) / np.abs(tabulation[pivot_row_idx])
/home/akila/semester_8/OR/helper.py:148: RuntimeWarning: invalid value encountered in divide
  ratio_row = np.abs(tabulation[0]) / np.abs(tabulation[pivot_row_idx])


**How to find the Maximum Change in the fully-utilized resource 1 such that the production mix is not going to change?**

In [20]:
-optimal_tabulation[:, -1] / contribution_margin_matrix[:, 1]

array([-53.46153846,  -5.        ,   5.32786885,  18.33333333])

**A new constraint is added...**

In [67]:
import pandas as pd

def save_tabulation(tabulation, all_vars, basic_vars, file_name):

    if not file_name.endswith('.csv'): file_name += '.csv'

    columns_labels = copy.deepcopy(all_vars)
    columns_labels.append("sol")

    DF = pd.DataFrame(tabulation, index=basic_vars, columns=columns_labels)
    DF.to_csv(file_name)

def load_tabulation(file_name):

    if not file_name.endswith('.csv'): file_name += '.csv'

    DF = pd.read_csv(file_name, index_col=0)  # Assuming the row labels are stored in the first column
    tabulation = DF.values
    basic_vars = DF.index.tolist()
    all_vars = DF.columns.tolist()

    return tabulation, all_vars, basic_vars

In [68]:
save_tabulation(optimal_tabulation, var_symbol_arr, basic_var_symbol_arr, 'opt_table')